# **EE4211               Question 3**

# **Question 3.1**
> #### At this point, you understand the data quite well. Carry out the analysis you proposed in your group project proposal. You should use the dataset given but you may also use additional datasets to supplement your analysis, look at unaggregated data, etc. Please be sure to justify why the analysis is useful and interesting in the context of a data science project. Note that you are not limited to the initial proposal and are free to expand on it.

### **Clustering data pre-processing**

In [ ]:
import pandas as pd
Data = pd.read_excel('Car_Park_Slots.xlsx', sheet_name = None)
sheet_names = list(Data.keys())[1:]

In [ ]:
import math
def get_data(sheet_name):
    data = Data[sheet_name]
    data = data[data.Type=='C'] # get the data that type == c
    data.loc[:, 'Rate'] = data['Available'] / data['Total']
    data.dropna(inplace=True)
    data.loc[data.Rate > 1, 'Rate'] = 1
    return data

AvailableRate = []
Dict = {}
count = 0
data = get_data(sheet_names[0])

for row in data.index:
    if math.isnan(data.loc[row]['Rate']) == False:
        Dict[data.loc[row]['Name']] = count
        count += 1
        AvailableRate.append([data.loc[row]['Rate']])

for i in range(1, len(sheet_names)):
    data = get_data(sheet_names[i])
    for row in data.index:
        if data.loc[row]['Name'] in Dict and math.isnan(data.loc[row]['Rate']) == False:
            AvailableRate[Dict[data.loc[row]['Name']]].append(data.loc[row]['Rate'])
    for j in range(len(AvailableRate)):
        if len(AvailableRate[j]) <= i:
            AvailableRate[j].append(AvailableRate[j][-1])

In [ ]:
# Avaialbe rate contains the availabilities of each parking lot for the past 744 time nodes.
CarParksName = []
AvailableRateUpd = []
for name, id in Dict.items():
    check = False
    val0 = AvailableRate[id][0]
    for val in AvailableRate[id]:
        if val0 != val:
            check = True
            break
    if check:
        CarParksName.append(name)
        AvailableRateUpd.append(AvailableRate[id])

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import metrics
import matplotlib.pyplot as plt

x = np.array(AvailableRateUpd)

for i in range(2, 11):
    k_means = KMeans(n_clusters=i, random_state=10)
    k_means.fit(x)

    y_predict = k_means.predict(x)
    print("clusters : " + str(i), end = "")
    # plt.show()
    # print(k_means.predict((x[:30,:])))
    # print(k_means.cluster_centers_)
    # print(k_means.inertia_)
    print(" silhouette_score : " + str(metrics.silhouette_score(x,y_predict)))


clusters : 2 silhouette_score : 0.398189151107227
clusters : 3 silhouette_score : 0.36156209549514334
clusters : 4 silhouette_score : 0.3303543648593959
clusters : 5 silhouette_score : 0.33384719792281514
clusters : 6 silhouette_score : 0.29413126288575775
clusters : 7 silhouette_score : 0.30218335365404053
clusters : 8 silhouette_score : 0.2720418518680833
clusters : 9 silhouette_score : 0.27626677523551546
clusters : 10 silhouette_score : 0.277635660159477


In [ ]:
from numpy import savetxt
k_means = KMeans(n_clusters=2, random_state=10)
k_means.fit(x)
y_predict = k_means.predict(x)

model_i = pd.DataFrame(y_predict, columns = ['model'])

carpark_ = []
for i in CarParksName:
    carpark_.append(i)
carpark_ = pd.DataFrame(carpark_, columns = ['Name'])

print(model_i)
print(carpark_)

In [ ]:
import csv

with open('NameandClass.csv', 'w', encoding='utf-8') as file_obj:
    writer = csv.writer(file_obj)
    writer.writerow(['Name', 'Class'])
    for i in range(len(y_predict)):
        writer.writerow([CarParksName[i], y_predict[i]])

center = k_means.cluster_centers_
with open('ClusterCenter.csv', 'w', encoding='utf-8') as file_obj:
    writer = csv.writer(file_obj)
    writer.writerow(['Class 0', 'Class 1'])
    for i in range(len(center[0])):
        writer.writerow([center[0][i], center[1][i]])

#### Predicting the available parking spaces around the destination at a given moment is extremely useful in our daily life. When going to an unfamiliar place, you are highly unconfident with successful parking since you need to know how many parking lots are left around the destination. Thus, accurate prediction results will save commuting time and make your day. 

#### As our project proposal suggested, we would like to predict the future empty rate for EVERY carpark. Since modelling one regressor for every carpark is over-expensive, we first clustered the data set (using October 2021 dataset) into clusters by using the K-means clustering algorithm. K-means clustering is an easy but efficient unsupervised clustering algorithm that can properly group samples together. In the clustering process, every carpark has the following features: 1. its name; 2. its empty rate over the past 744 samples. Those two features are sufficient for our requirements since we are only focusing on the open rate for each carpark and trying to group them by their behaviour on the empty rate. By looking at the silhouette scores, we can tell how effective the clustering is. The silhouette score is a coefficient that indicates how cohesive the sample is in a cluster. When silhouette scores approach 1, the samples are clustered reasonably. When silhouette scores approach -1, the samples are clustered unreasonably. By running a series of test on the number of clusters from 2 to 10, we concluded that the silhouette scores is the highest; thus, the samples are best clustered when the number of clusters is 2.



In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import metrics
import matplotlib.pyplot as plt

x = np.array(AvailableRateUpd)

k_means = KMeans(n_clusters=2, random_state=10)
k_means.fit(x)

y_predict = k_means.predict(x)
print("clusters : " + str(i), end = "")
plt.show()
print(k_means.predict((x[:30,:])))
print(k_means.cluster_centers_)
print(k_means.inertia_)
print(" silhouette_score : " + str(metrics.silhouette_score(x,y_predict)))


clusters : 10[1 0 1 0 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1]
[[0.54766771 0.54216758 0.53976819 ... 0.57436899 0.56519102 0.55341931]
 [0.24084325 0.2391177  0.23718832 ... 0.26004434 0.26273662 0.25418812]]
36156.703667259324
 silhouette_score : 0.398189151107227


#### To make real-time carpark calculation and navigation possible, we introduced carparks' [geographic location](https://data.gov.sg/dataset/hdb-carpark-information) by combining carparks' geolocation and car parks name into our system. The destination navigation feature is possible with the help of [Onemap](https://www.onemap.gov.sg/docs/) (developed by the Singapore Land Authority) application programming interface.

#### To project cluster results to the map of Singapore, We use red and blue dots to visualize the two clusters of parking lots in terms of geographic coordinates. The graph shows that the two kinds of parking lots are interleaved together, which indicates that the clustering result is not related to geographical coordinates but the variation pattern of the empty rate. The results also show that drivers tend to choose the desired carpark randomly.

![picture](./images/Carpark_map.png)

# **Question 3.2**
> #### Based on the insights derived from the analysis, suggest a practical action that can be taken (i.e., an action that can be taken to benefit society. Do not suggest actions such as hyperparameter tuning here).

In [ ]:
from urllib.request import Request, urlopen
import json
import urllib
import math
from queue import PriorityQueue

In [ ]:
def extract(offset=0):
    url = 'https://data.gov.sg/api/action/datastore_search?offset=%d&resource_id=139a3035-e624-4f56-b63f-89ae28d4ae4c' % (offset)
    headers = {'User-Agent': 'Mozilla/5.0'}
    req = urllib.request.Request(url,headers=headers)
    response = urllib.request.urlopen(req)
    hjson = json.loads(response.read())

    return hjson

In [ ]:
def getDistance(x,y,record):
    x1 = float(record['x_coord'])
    y1 = float(record['y_coord'])

    return math.sqrt((x-x1)**2+(y-y1)**2)

In [ ]:
# Retreive the nearest k carparks by entering the lat and long and lat coordination.
# return: list that contains the nearest k carparks info
# x: long coordinate
# y: lat  coordinate
# k: Number of carparks

def getNearestK(x, y, k):
    offset = 0
    pq = PriorityQueue()
    while True:
        hjson = extract(offset)
        if len(hjson['result']['records']) == 0:
            break 
        for record in hjson['result']['records']:
            dist = -getDistance(x, y, record)
            if pq.qsize() == k:
                top, topRecord = pq.get()
                if(top>dist):
                    pq.put((top,topRecord))
                else:
                    pq.put((dist,record))
            else:
                pq.put((dist,record))
        offset += 100

    ans = []
    while (not pq.empty()):
        top = pq.get()
        ans.append((-top[0],top[1]))
    return ans

In [ ]:
x = getNearestK(0, 0, 5)

In [ ]:
# Get the nearest K carparks' information.
# return:           dataframe that contains the nearest k carparks address, carpark name, address, and cluster tag
# nearest_carparks: return values from getNearestK function;
# k:                Number of carparks

def getNearestK_info(nearest_carparks, k):

  nearest_carparks_info = pd.DataFrame(columns=['car_park_no','address','cluster_tag'])
  cluster_tag = pd.read_csv("NameandClass.csv")

  for i in range(k):
    car_park_no = nearest_carparks[i][1]['car_park_no']
    carpark_ctag = cluster_tag[cluster_tag['Name']==car_park_no]['Class'].tolist()
    if len(carpark_ctag) == 0:#Drop the carpark if no empty rate applicatble
      continue
    carpark_info = pd.DataFrame([[car_park_no,nearest_carparks[i][1]['address'],carpark_ctag[0]]], columns=['car_park_no','address','cluster_tag'])
    
    nearest_carparks_info = nearest_carparks_info.append(carpark_info,ignore_index=True)

  return nearest_carparks_info

In [ ]:
nearestK_carparks_info = getNearestK_info(x, 5)
print(nearestK_carparks_info)

  car_park_no                             address cluster_tag
0        J62M       BLK 901 JURONG WEST STREET 91           0
1        JM24       BLK 655 JURONG WEST STREET 61           0
2         C38  BLK 715-717 CLEMENTI WEST STREET 2           1
3        JM28           BLK 990 JURONG WEST ST 93           0


### **Regressors Training**

#### To train the regreesors that can predict carpark's empty rate in futre, we used the history data from 2021 oct as our train and test dataset.

#### After the K-means clustering, every carpark has a unique tag to determine which group it belongs to and which regressor it should use to predict future empty rates. We trained two decision tree regression models and two support vector regressor models to predict carparks' empty rate. **The features we used in regressors are 1. Departure date; 2. Expected arrival time; 3. Workday/Weekend or Holiday. The label is the average empty rate of the selected cluster at the given time.** According to our test results, those two models accurately predict carparks' emtpy rate. The R^2 scores are 0.964 and 0.967 on our test data. With this application, drivers can avoid driving from one carpark to another to find an available parking space after arriving at the place. This application will ease drivers' pain of searching for parking lots and improve road utilization.


In [ ]:
import pandas as pd
import sklearn
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
%matplotlib inline 

In [ ]:
# Get training and testing data ready
import datetime

aggregated = [[] for i in range(744)]
year = 2021
month = 10
for day in range(1, 32):
    for hour in range(24):
        aggregated[(day-1)*24+hour].append(day - 92)
        aggregated[(day-1)*24+hour].append(hour)
        if datetime.date(year, month, day).weekday() >= 5:
            aggregated[(day-1)*24+hour].append(1)
        else:
            aggregated[(day-1)*24+hour].append(0)

In [ ]:
# Train test data split; 70% training data; 30% testing data
from sklearn.model_selection import train_test_split

dataset = pd.DataFrame(aggregated)
dataset_cluster0 = pd.concat([dataset,pd.read_csv('ClusterCenter.csv')['Class 0']],axis = 1)
dataset_cluster1 = pd.concat([dataset,pd.read_csv('ClusterCenter.csv')['Class 1']],axis = 1)


x_train_0, x_test_0, y_train_0, y_test_0 = train_test_split(dataset_cluster0.loc[:,:2], dataset_cluster0['Class 0'], test_size=0.3)
x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(dataset_cluster1.loc[:,:2], dataset_cluster1['Class 1'], test_size=0.3)

In [ ]:
# DTR for 2 models
model = [DecisionTreeRegressor(), DecisionTreeRegressor()]
model[0].fit(x_train_0, y_train_0)
model[1].fit(x_train_1, y_train_1)

print(model[0].score(x_test_0, y_test_0))
print(model[1].score(x_test_1, y_test_1))

0.963956591964612
0.966709076098193


In [ ]:
# SVR for 2 models
from sklearn.preprocessing import RobustScaler

rbX = RobustScaler()

x_train_scaler_0 = rbX.fit_transform(x_train_0)
x_test_scaler_0 = rbX.transform(x_test_0)

x_train_scaler_1 = rbX.fit_transform(x_train_1)
x_test_scaler_1 = rbX.transform(x_test_1)

model_SVR = [SVR(kernel = 'rbf'), SVR(kernel = 'rbf')]
model_SVR[0].fit(x_train_scaler_0, y_train_0)
model_SVR[1].fit(x_train_scaler_1, y_train_1)

print(model_SVR[0].score(x_test_scaler_0, y_test_0))
print(model_SVR[1].score(x_test_scaler_1, y_test_1))

-0.03845521242917194
0.03464442403106449


#### **According to the R^2 results, DTR has a much better performance than SVR, thus we will focusing on DTR to do the parameter tuning**

In [ ]:
# Hyper parameter tuning
dtr_performance0 = []
dtr_performance1 = []
range_max_depth = np.arange(3,30, 1)


for i in range(3,30,1):
  model = [DecisionTreeRegressor(max_depth = i), DecisionTreeRegressor(max_depth = i)]
  model[0].fit(x_train_0, y_train_0)
  model[1].fit(x_train_1, y_train_1)

  dtr_performance0.append(model[0].score(x_test_0, y_test_0))
  dtr_performance1.append(model[1].score(x_test_1, y_test_1))

dtr_performance0 = np.array(dtr_performance0)
dtr_performance1 = np.array(dtr_performance1)
print("When max_Depth =  {}:".format(range_max_depth[dtr_performance0.argmax()]))
print("DTR for cluster 0 has the best performance {}".format(dtr_performance0[dtr_performance0.argmax()]))
print("When max_Depth =  {}:".format(range_max_depth[dtr_performance1.argmax()]))
print("DTR for cluster 1 has the best performance {}".format(dtr_performance1[dtr_performance1.argmax()]))

When max_Depth =  15:
DTR for cluster 0 has the best performance 0.9639712652333822
When max_Depth =  20:
DTR for cluster 1 has the best performance 0.9668871464930775


#### Our testing revealed that the models perform at their peak when "max depth" is 15 and 20, respectively. Because the tree will automatically grow until all of its leaves are pure or contain fewer samples than min samples split. However, because the leaf nodes are too pure, the model overfits and loses its capacity to generalize to unseen data. To avoid the model from overfitting, we change the max depth in this regard.

In [ ]:
model = [DecisionTreeRegressor(max_depth = 15), DecisionTreeRegressor(max_depth = 20)]
model[0].fit(x_train_0, y_train_0)
model[1].fit(x_train_1, y_train_1)

print(model[0].score(x_test_0, y_test_0))
print(model[1].score(x_test_1, y_test_1))

0.9639731120236347
0.9666633418953358


  <br/>
    <br/>
      <br/>
        <br/>
          <br/>
            <br/>
    <br/>
      <br/>
        <br/>
          <br/>
              <br/>
          <br/>

### **Demo**

In [ ]:
# Demonstrate how to run neaest K first and regressor to get final output empty rate
x=getNearestK(100,95,10) #find the nearest 10 carpark at lag = 100; lag = 95
nearestK_carparks_info = getNearestK_info(x,5) #get the cluster tags and address information in plain english

In [ ]:
# Date from 2022.01.01, Hour, Weekday or not weekday
queryDate = [[315, 11, 1]]  # Use this time to as the arrive time
empty_rates = []
for i in range(nearestK_carparks_info.shape[0]):
  tag = nearestK_carparks_info.iloc[i]['cluster_tag']
  empty_rates.append(model[tag].predict(queryDate)[0]) # Select model according to the cluster tag of each carpark and predict

empty_rates = pd.DataFrame(empty_rates,columns =['empty rate'])
nearestK_carparks_info = pd.concat([nearestK_carparks_info,empty_rates],axis =1)
print(nearestK_carparks_info)

  car_park_no                                     address cluster_tag  \
0        TJ27                    BLK 151/154 YUNG HO ROAD           1   
1         J62           BLK 902/908 JURONG WEST STREET 91           1   
2        JM20               BLK 657 JURONG WEST STREET 65           0   
3         C37  BLK 720-727,730-731 CLEMENTI WEST STREET 2           1   
4        J62M               BLK 901 JURONG WEST STREET 91           0   

   empty rate  
0    0.276897  
1    0.276897  
2    0.572074  
3    0.276897  
4    0.572074  


### **User interface mockup**

A mockup user interface of the product shown below. We showed how to use the product and the process in the form of pictures and text.

> 1. Open the Application, the location function will locate user's current location;<br/>
2. Key in the destination;<br/>
3. Key in the time of departure and the traveling method;<br/>
4. Display the number of available parking spaces in the parking lot that around the destination address.

![picture](./images/merged.png)

# **Appendix**

## Code for generating the scatter on SINGAPORE map

In [ ]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
f1 = pd.read_csv('hdb-carpark-information.csv')
f2 = pd.read_csv('NameandClass.csv')

f1 = f1.rename(columns={'car_park_no':'Name','x_coord':'x','y_coord':'y'})

In [ ]:
df = pd.merge(f1,f2)
df1 = df.loc[df.Class==1]
df0 = df.loc[df.Class==0]

px1 = df1['x'].tolist()
py1 = df1['y'].tolist()
px0 = df0['x'].tolist()
py0 = df0['y'].tolist()

from PIL import Image
fig = plt.figure(figsize=(12,6))
# plt.imshow(image)
plt.scatter(px1,py1,c='r',s=10)
plt.scatter(px0,py0,c='b',s=10)